In [2]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.neighbors import NearestNeighbors

df_profile = pd.read_csv('C:\\Users\\SUVARNA\\Food Recommendation\\Data Processing\\user_Profiles.csv')
#df_activity = pd.read_csv('C:\\Users\\SUVARNA\\Food Recommendation\\Data Processing\\recent_activity.csv')
df_profile.head(3)

,User_Id,Veg_Non,Nutrient,Disease,Diet
0,User_1,veg,selenium,goitre diabeties pregnancy,low_fat_diet high_fiber_diet ketogenic_diet v...
1,User_2,veg,vitamin_d,goitre,high_fiber_diet ketogenic_diet
2,User_3,veg,protien,heart_disease pregnancy,high_fiber_diet high_protien_diet
